In [1]:
import os 
import numpy as np
import skimage
import tifffile as tifffile
from skimage.transform import downscale_local_mean
from skimage.filters import rank_order
from skimage.filters import median
from skimage.filters import threshold_multiotsu as multiotsu
import csv
import nibabel as nib

In [2]:
# FK added 19/05/2023
# small function to avoid importing .DS_Store on macos
def listdir_nohidden(path):
    return( sorted([f for f in os.listdir(path) if not f.startswith('.')]) )

In [10]:
def thresh_brains(brain):
    brain_8bit = brain.astype('uint8')
    #thresh = multiotsu(brain_8bit,classes=2,nbins=256)
    #threshed = brain_8bit>thresh*2.5 # autothreshold value (thresh) * multiplier of your chlice
    threshed = brain_8bit>3*5  #force threshold value of your choic
    result = median(threshed)
    return result    

In [8]:
def mask_process(mask_path, brain):
    brain_pixels = []
    mask_pixels = []
    ratio_pixels =[]
    
    for files in listdir_nohidden(mask_path): # FK nohidden version otherwise tries to import .DSStore
        mask = tifffile.imread(mask_path + files) #open one mask at a time; # FK 19/05/2023 deleted + '//' + as using on macos
        result = thresh_brains(brain)
        brain_m = np.bitwise_and(result, mask) #bitwise and operation on mask and brain
        count_pixels = np.count_nonzero(brain_m) #number of pixels above 0
        mask_pix = np.count_nonzero(mask) #number of pixels in mask
        ratio = (count_pixels/mask_pix)*100 
        brain_pixels.append(count_pixels)
        mask_pixels.append(mask_pix)
        ratio_pixels.append("%.2f" % ratio) #round to 2 decimal places
        
    return brain_pixels, mask_pixels, ratio_pixels

In [7]:
#change: brain_dir (folder with brains of interest)

mask_dir = "/Users/francoiskroll/Dropbox/ZFAD/sorl1HCR/zbbMasks/" #mask directory

# – RUN1 –
# brain_dir = "/Users/francoiskroll/Dropbox/ZFAD/sorl1HCR/tphSCR_maxRescaled/" #data directory

# – RUN2 –
# brain_dir = "/Users/francoiskroll/Dropbox/ZFAD/sorl1HCR/tphSORL1_maxRescaled/" #data directory

# – RUN3 –
# brain_dir = "/Users/francoiskroll/Dropbox/ZFAD/sorl1HCR/sertSCR_maxRescaled/" #data directory

# – RUN4 –
# brain_dir = "/Users/francoiskroll/Dropbox/ZFAD/sorl1HCR/sertSORL1_maxRescaled/" #data directory

brains=[]

for files in listdir_nohidden(brain_dir): # FK 19/05/2023 was os.listdir(brain_dir), changed to def listdir_nohidden(brain_dir): because on macos tries to import .DS_Store file
    print(files)
    brain = tifffile.imread(brain_dir + files) # FK 19/05/2023 deleted + '//' + as using on macos
    rescale_brain = downscale_local_mean(brain, (2, 2, 2)) # scale brains down to mask size
    rescale_brain = rescale_brain.astype('uint8')
    brains.append(rescale_brain)
    %xdel brain

F10_010b_SCR_tph.tif
F1_010b_SCR_tph.tif
F2_010b_SCR_tph.tif
F3_010b_SCR_tph.tif
F4_010b_SCR_tph.tif
F5_010b_SCR_tph.tif
F6_010b_SCR_tph.tif
F7_010b_SCR_tph.tif
F8_010b_SCR_tph.tif
F9_010b_SCR_tph.tif


In [11]:
pixel_mask_result = [] #list of dictionaries for each brain: {mask name: pixel number}
mask_names = listdir_nohidden(mask_dir) # listdir_nohidden(mask_dir)

i=1

for brain in brains:
    print('\n brain num:', i)
    brain_pix, mask_pix, ratio = mask_process(mask_dir, brain) # calling function
    ziptogether = zip(mask_names, brain_pix, mask_pix, ratio)
    pixel_mask_list = list(ziptogether)
    pixel_mask_result.append(pixel_mask_list)
    i=i+1
    


 brain num: 1

threshed

[[[False False False ... False False False]
  [False False False ... False False False]
  [False False False ... False False False]
  ...
  [False False False ... False False False]
  [False False False ... False False False]
  [False False False ... False False False]]

 [[False False False ... False False False]
  [False False False ... False False False]
  [False False False ... False False False]
  ...
  [False False False ... False False False]
  [False False False ... False False False]
  [False False False ... False False False]]

 [[False False False ... False False False]
  [False False False ... False False False]
  [False False False ... False False False]
  ...
  [False False False ... False False False]
  [False False False ... False False False]
  [False False False ... False False False]]

 ...

 [[False False False ... False False False]
  [False False False ... False False False]
  [False False False ... False False False]
  ...
  [False False

KeyboardInterrupt: 

In [7]:
#change: NT_geno (neurotransmitter and genotype), writefile (save to folder)

writefile = "/Users/francoiskroll/Dropbox/ZFAD/sorl1HCR/activeVoxels_v3/" # results file directory

# NT_geno = "SCR_tph" # suffix for control or treated or gene expression
# NT_geno = "SORL1_tph"
# NT_geno = "SCR_sert"
# NT_geno = "SORL1_sert"

brain_num = 1  

headers = ['Anatomy', 'Active Voxels', 'Total Voxels', 'Ratio (%)']

for pixels in pixel_mask_result:
    pix_data = open(writefile + NT_geno + str(brain_num) + '.csv', 'w', newline='') # FK 19/05/2023 deleted + '//' + as using on macos
    brain_num += 1
    writer = csv.writer(pix_data)
    writer.writerow(headers)
    
    for ind, data in enumerate(pixels):
        writer.writerow(data)
        
    pix_data.close()